In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import zipfile
from sklearn.metrics import mean_squared_error

In [2]:
zip_path = '../00_Data_Bases/df_recommendationsystem.zip' 
csv_filename = 'df_recommendationsystem.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df_cl5 = pd.read_csv(f)

In [3]:
df_cl5.head()

,department_babies,department_bakery,department_beverages,department_breakfast,department_bulk,department_canned goods,department_dairy eggs,department_deli,department_dry goods pasta,department_frozen,...,department_other,department_pantry,department_personal care,department_pets,department_produce,department_snacks,max_order,Clusters_5,Customer_Type,Clusters_5_1_PCA
0,0,1,1,2,0,2,31,7,3,5,...,0,4,1,0,18,4,38,4,Consumidor recurrente y especializado,4
1,0,3,4,0,0,0,6,1,4,0,...,0,1,0,0,24,5,90,2,Comprador ocasional que realiza compras divers...,1
2,0,0,1,0,0,0,6,0,0,0,...,0,4,0,0,3,0,3,3,Compradores Mensuales,0
3,0,0,2,0,0,0,1,0,0,0,...,0,1,0,0,3,1,18,1,Compradores Leales,2
4,2,0,3,1,0,0,4,0,0,0,...,0,0,9,0,5,2,8,0,Compradores Nuevos o Infrecuentes,3


In [9]:
zip_path = '../00_Data_Bases/Supermercado_mod.zip' 
csv_filename = 'Supermercado_mod.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df_mod = pd.read_csv(f)

In [10]:
df_mod.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name,day,order_time_list,max_order,order_number_group
0,1201011,147243,14,0,16,3.0,94,1,0,7,beverages,tea,Monday,Afternoon,38,21-40 order
1,1201011,147243,14,0,16,3.0,83,2,0,4,produce,fresh vegetables,Monday,Afternoon,38,21-40 order
2,1201011,147243,14,0,16,3.0,83,3,1,4,produce,fresh vegetables,Monday,Afternoon,38,21-40 order
3,1201011,147243,14,0,16,3.0,24,4,1,4,produce,fresh fruits,Monday,Afternoon,38,21-40 order
4,1201011,147243,14,0,16,3.0,120,5,1,16,dairy eggs,yogurt,Monday,Afternoon,38,21-40 order


In [11]:
df_cl5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98918 entries, 0 to 98917
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   department_babies           98918 non-null  int64 
 1   department_bakery           98918 non-null  int64 
 2   department_beverages        98918 non-null  int64 
 3   department_breakfast        98918 non-null  int64 
 4   department_bulk             98918 non-null  int64 
 5   department_canned goods     98918 non-null  int64 
 6   department_dairy eggs       98918 non-null  int64 
 7   department_deli             98918 non-null  int64 
 8   department_dry goods pasta  98918 non-null  int64 
 9   department_frozen           98918 non-null  int64 
 10  department_household        98918 non-null  int64 
 11  department_international    98918 non-null  int64 
 12  department_meat seafood     98918 non-null  int64 
 13  department_missing          98918 non-null  in

In [12]:
df_mod.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895159 entries, 0 to 1895158
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   department_id           int64  
 10  department              object 
 11  product_name            object 
 12  day                     object 
 13  order_time_list         object 
 14  max_order               int64  
 15  order_number_group      object 
dtypes: float64(1), int64(10), object(5)
memory usage: 231.3+ MB


In [13]:
df_mod = df_mod.merge(df_cl5[['user_id', 'Clusters_5']], on='user_id', how='left')

df_mod.head()

KeyError: "['user_id'] not in index"

In [9]:
# Verificar unicidad de Clusters_4 para cada user_id
unique_clusters = df_cl4.groupby('user_id')['Clusters_4'].nunique()

# Filtrar los casos donde haya más de un valor único de Clusters_4 para un user_id
non_unique_users = unique_clusters[unique_clusters > 1]

# Mostrar los resultados
if not non_unique_users.empty:
    print("Los siguientes user_id tienen más de un valor único en Clusters_4:")
    print(non_unique_users)
else:
    print("Cada user_id tiene un único valor en Clusters_4.")


Cada user_id tiene un único valor en Clusters_4.


In [10]:
# Contar la frecuencia de valores únicos en la columna Clusters_4
cluster_counts = df_cl4['Clusters_4'].value_counts()

# Mostrar los resultados
print(cluster_counts)


Clusters_4
1    45352
3    27232
2    20262
0     6072
Name: count, dtype: int64


In [11]:
# Filtrar filas donde Clusters_4 es igual a 1
df_cl4_1 = df_cl4[df_cl4['Clusters_4'] == 1]

In [12]:
df_cl4_1.to_csv('../00_Data_Bases/Cluster4_1_items.csv', index=False)

zip_filename = '../00_Data_Bases/Cluster4_1_items.zip'
csv_filename = '../00_Data_Bases/Cluster4_1_items.csv'

with zipfile.ZipFile(zip_filename, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename, arcname='Cluster4_1_items.csv')